<a href="https://colab.research.google.com/github/viniciusrpb/cic0193_machinelearning/blob/main/cap9_4_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential
from keras.layers import Dense,MaxPooling2D,Activation,Flatten,Conv2D,BatchNormalization,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from sklearn.metrics import classification_report
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp -r "/content/drive/My Drive/leafs" "leafs"

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255.)

train_set = train_datagen.flow_from_directory("leafs/train",
                                              subset='training',
                                              target_size=(224,224),
                                              class_mode="categorical",
                                              batch_size=32)

validation_set = train_datagen.flow_from_directory("leafs/train",
                                              subset='validation',
                                              target_size=(224,224),
                                              class_mode="categorical",
                                              batch_size=32)

test_set = test_datagen.flow_from_directory("leafs/test",
                                            target_size=(224,224),
                                            class_mode="categorical",
                                            batch_size=32)



Found 160 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [5]:
kernel_dimensions = (5,5)
tam_strides = (1,1)
number_filters = 50
number_of_neurons = 30

num_classes = 4

Transfer Learning

In [6]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape=(224,224,3),
                                include_top=False,
                                weights='imagenet')

87924736/87910968 [==============================] - 2s 0us/step


In [7]:
for layer in pre_trained_model.layers:
  layer.trainable=False

In [18]:
ans = layers.Flatten()(pre_trained_model.output)

ans = layers.Dense(512,activation="relu")(ans)

ans = layers.Dropout(0.2)(ans)

ans = layers.Dense(num_classes,activation="softmax")(ans)

model_tl = Model(pre_trained_model.input,ans)

In [9]:
#model_tl.summary()

In [23]:
model_tl.compile(optimizer=SGD(learning_rate=0.1),
                 loss = "categorical_crossentropy",
                 metrics=["accuracy"])

In [24]:
history_fine = model_tl.fit(train_set,
                            epochs=20,
                            validation_data=validation_set
                            #initial_epoch=history.epoch[-1],
                            )

Epoch 1/20
5/5 [==============================] - 26s 5s/step - loss: 1.3874 - accuracy: 0.1937 - val_loss: 1.3862 - val_accuracy: 0.2500
Epoch 2/20
5/5 [==============================] - 22s 4s/step - loss: 1.3869 - accuracy: 0.2500 - val_loss: 1.3862 - val_accuracy: 0.2500
Epoch 3/20
5/5 [==============================] - 22s 4s/step - loss: 1.3871 - accuracy: 0.2250 - val_loss: 1.3862 - val_accuracy: 0.2500
Epoch 4/20
5/5 [==============================] - 22s 4s/step - loss: 1.3882 - accuracy: 0.2000 - val_loss: 1.3862 - val_accuracy: 0.2500
Epoch 5/20
5/5 [==============================] - 22s 4s/step - loss: 1.3873 - accuracy: 0.2313 - val_loss: 1.3862 - val_accuracy: 0.2500
Epoch 6/20
5/5 [==============================] - 22s 4s/step - loss: 1.3876 - accuracy: 0.2500 - val_loss: 1.3862 - val_accuracy: 0.2500
Epoch 7/20
5/5 [==============================] - 22s 4s/step - loss: 1.3865 - accuracy: 0.2375 - val_loss: 1.3861 - val_accuracy: 0.2500
Epoch 8/20
5/5 [==================

In [25]:
y_prob = model_tl.predict(test_set)
y_pred = np.argmax(y_prob,axis=1)

print(classification_report(test_set.classes,y_pred))

              precision    recall  f1-score   support

           0       0.25      1.00      0.40        25
           1       1.00      0.04      0.08        25
           2       0.00      0.00      0.00        25
           3       0.00      0.00      0.00        25

    accuracy                           0.26       100
   macro avg       0.31      0.26      0.12       100
weighted avg       0.31      0.26      0.12       100



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
